# Libraries

In [ ]:
!apt install ffmpeg

In [ ]:
#pip install python-ffmpeg

In [2]:
#import ffmpeg
import cv2
import os
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Folders & Files

In [3]:
mkdir exported

In [17]:
!rm ./exported/*.png
#rm exported/*.jpg

# Image Processing

DVD: 420x720

720p: 720x1280

1080p: 1080x1920

4K: 2160x3840

8K: 4320x7680

In [18]:
# Master
'''
(height, width)
  = (480, 720)
  = (720, 1280)
  = (1080, 1920)
  = (1440, 2560)
  = (1920, 2880) # Surface
  = (2160, 3840)
  = (4320, 7680)
'''

height = 4320 #2160 #480
width = 7680 #3840 #720

hBand = 16

unit = 2**13
qty = int(2**16/unit) #2**(16-13)

i = 0

for stt in [0]: # [0, 1]
  for b in [0] + [(i+1) * unit for i in range(qty)]:
    for g in [0] + [(i+1) * unit for i in range(qty)]:
      for r in [0] + [(i+1) * unit for i in range(qty)]:
        hdr = np.zeros((height, width, 3), np.uint16)
        hdr[:,:,:] = (2**16-1)*stt//1 # 0:shadow, 1:tint
        if (b == 2**16 and (g == 0 or g == 2**16)) or \
          (g == 2**16 and (r == 0 or r == 2**16)) or \
          (r == 2**16 and (b == 0 or b == 2**16)) or \
          (b == 0 and g == 2**16) or \
          (g == 0 and r == 2**16) or \
          (r == 0 and b == 2**16):
          for h in range(height):
            for w in range(width):
              if b != 0:
                hdr[h, w, 0] = int(
                    (b-1) / (hBand * width - 1) 
                    * (w + 1 + width * (h//(height//hBand)))
                    )
              if g != 0:
                hdr[h, w, 1] = int(
                    (g-1) / (hBand * width - 1) 
                    * (w + 1 + width * (h//(height//hBand)))
                    )
              if r != 0:
                hdr[h, w, 2] = int(
                    (r-1) / (hBand * width - 1) 
                    * (w + 1 + width * (h//(height//hBand)))
                    )
          #cv2.imwrite(f'exported/{stt}_b{b}g{g}r{r}_unit{unit}qty{qty}.png', hdr)
          #cv2.imwrite('exported/%01d_b%05dg%05dr%05d_unit%05dqty%05d.png' % (stt, b, g, r, unit, qty), hdr)
          cv2.imwrite('exported/%03d.png' % (i), hdr)
          i += 1

## Image to Video
https://en.wikibooks.org/wiki/FFMPEG_An_Intermediate_Guide/image_sequence

https://shotstack.io/learn/use-ffmpeg-to-convert-images-to-video/

In [ ]:
#master
!ffmpeg \
  -y \
  -r 1 \
  -i ./exported/%03d.png \
  ./exported/image2video.mp4

In [ ]:
#Master 100M
# unable to hundle 8K image
!ffmpeg \
  -y \
  -i ./exported/%03d.png \
  -pix_fmt yuv444p10le \
  -vb 100000k \
  ./exported/image2video_vb100M.mp4

In [ ]:
# 1G bps
!ffmpeg \
  -y \
  -i ./exported/%03d.png \
  -pix_fmt yuv444p10le \
  -vb 1000000k \
  ./exported/image2video_vb100M.mp4

In [ ]:
!ffmpeg \
  -y \
  -framerate 1 \
  -i %03d.png \
  -r 6 \
  -vf format=yuv444p10le \
  -vcodec libx265 \
  -crf 20 \
  -tag:v hvc1 \
  -color_primaries bt2020 \
  -color_trc smpte2084 \
  -colorspace bt2020nc \
  -color_range tv \
  hdr_6fps.mov